# A journey into Convolutional Neural Network visualisation 

There is one famous urban legend about computer vision. Around the 80s, the US military wanted to use neural networks to automatically detect camouflaged enemy tanks. They took a number of pictures of trees without tanks and then pictures with the same trees with tanks behind them.
The results were impressive. So impressive that the army wanted to be sure the net had correctly generalized. They took new pictures of woods with and without tanks and they showed them again to the network. This time, the model performed terribly, it was not able to discriminate between pictures tanks behind woods and just trees. It turned out, that all the pictures with of plain woods were taken on a cloudy day while the ones with tanks on a sunny day! In reality, the network lean to recognize the weather, not the enemy tanks.

With this article we are going to see different techiniques to *understand* what it is going on inside a Convolutional Neural Network to making the same mistake made, ipotetically, by the US army.

The structure is the following:
 - Introduction of the most used techniques
 - Visualisation of common neural networks
 - Visualisation of uncommon neural networks
 - Conclusion
 
We are going to use [Pytorch](https://pytorch.org/) and not Tensorflow since Pytorch is a much better framework under all aspects, providing good practice a well written documentation, two main features the Google is missing. All the code can be found here, the visualisations classes where taken

# Weights Visualisation
The first straightfoward visualisation is just plot the weigths of a target Layer

In [1]:
class Base:
    def __init__(self, module, device):
        self.module, self.device = module, device
        self.handles = [] # handles stores the hooks
 
    def clean(self):
        [h.remove() for h in self.handles]

    def __call__(self, inputs, layer, *args, **kwargs):
        return inputs